In [2]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=vocabulary_size, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(paths, trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [4]:
paths = ['/content/sw-train.txt']
path = '/content/sw-train.txt'
file1 = open(path, "r")
text = file1.read()
len(text)
chars = sorted(list(set(text)))
vocabulary_size = len(chars)

In [5]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))

Total chars: 49


In [6]:
tokenizer = BPE_token()
tokenizer.bpe_train(paths)
tokenizer.save_tokenizer('.')

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 70.2 MB/s 


In [8]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained('.')

# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  n_embd=256,
  n_layer=12,
  n_head=8
)
# creating the model
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 9753600   
 r)                                                              
                                                                 
Total params: 9,753,600
Trainable params: 9,753,600
Non-trainable params: 0
_________________________________________________________________


In [9]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
    x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [10]:
examples = []
block_size = 40
step = 3
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, 3):
  examples.append(string_tokenized[i:i + block_size])


In [11]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [13]:
num_epoch = 3
history = model.fit(dataset,batch_size= 204800, epochs=num_epoch,validation_data=dataset)

Epoch 1/3
32717/32717 [==============================] - 2627s 80ms/step - loss: 1.7467 - accuracy: 0.4824 - val_loss: 1.4502 - val_accuracy: 0.5664
Epoch 2/3
32717/32717 [==============================] - 2606s 80ms/step - loss: 1.4325 - accuracy: 0.5698 - val_loss: 1.3418 - val_accuracy: 0.5950
Epoch 3/3
32717/32717 [==============================] - 2604s 80ms/step - loss: 1.3535 - accuracy: 0.5915 - val_loss: 1.2879 - val_accuracy: 0.6095


In [14]:
text = "hapo zamani za kale"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 100,
  num_beams = 5,
  temperature = 0.4,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)
print(tokenizer.decode(beam_output[0]))


hapo zamani za kale. hivyo, watakuaje? sikilize mwenyezi! tunashukru pia,ndugu,mtongoji,wimbo..mumeo
